In [59]:
%load_ext autoreload
%autoreload 2

from src.rxn_ctr_mcs import *
from src.utils import load_json, rxn_entry_to_smarts, rm_atom_map_num
from src.pathway_utils import get_reverse_paths_to_starting, create_graph_from_pickaxe
from src.post_processing import *

from minedatabase.pickaxe import Pickaxe
from minedatabase.utils import get_compound_hash

from rdkit.Chem import AllChem
import pickle


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
# Load processed expansion
starters = 'ccm_v0'
targets = 'hopa'
generations = 3

expansion_dir = '../data/processed_expansions/'
thermo_dir = '../data/thermo/'
fn = f"{starters}_to_{targets}_gen_{generations}_tan_sample_1_n_samples_1000.pk" # Expansion file name
rxns_path = expansion_dir + 'predicted_reactions_' + fn
paths_path = expansion_dir + 'paths_' + fn
pruned_dir = '../data/pruned_expansions/'

# Load reactions and paths
with open(rxns_path, 'rb') as f:
    pred_rxns = pickle.load(f)

with open(paths_path, 'rb') as f:
    paths = pickle.load(f)

# Load raw expansion object
pk = Pickaxe()
path = pruned_dir + fn
pk.load_pickled_pickaxe(path)

----------------------------------------
Intializing pickaxe object

Done intializing pickaxe object
----------------------------------------

Loading ../data/pruned_expansions/ccm_v0_to_hopa_gen_3_tan_sample_1_n_samples_1000.pk pickled data.
Loaded 65 compounds
Loaded 83 reactions
Loaded 3604 operators
Loaded 1 targets
Took 2.9580531120300293


In [61]:
print(len(pred_rxns))
for k,v in paths.items():
    print(k, len(v))

83
('malate', 'hopa') 29
('succinate', 'hopa') 7
('citrate', 'hopa') 10
('oxaloacetate', 'hopa') 24
('pyruvate', 'hopa') 26


In [62]:
pr_am_errors = [] # Track predicted rxn am errors
kr_am_errors = [] # Track known rxn am errors
alignment_issues = [] # Track substrate alignment issues
kekulize_issues = []
norm = 'max atoms' # Normalize MCS atom count by larger molecule

# Populate pred_rxns, known rxn prc-mcs slot
for x in range(len(pred_rxns.keys())):
    h = list(pred_rxns.keys())[x]
    rxn_sma1 = pred_rxns[h].smarts

    # Skip pred reactions that trigger RXNMapper atom mapping errors
    try:
        am_rxn_sma1 = atom_map(rxn_sma1)
    except:
        pr_am_errors.append(h)
        continue

    a = 0 # Number known rxns analyzed
    for z, kr in enumerate(pred_rxns[h].known_rxns):
        rxn_sma2 = kr[1]

        # Catch stoichiometry mismatches stemming from pickaxe, early post-processing
        if tuple([len(elt.split('.')) for elt in rxn_sma2.split('>>')]) != tuple([len(elt.split('.')) for elt in rxn_sma1.split('>>')]):
            print(x, z, 'stoich_error')
            continue

        # Skip pred reactions that trigger RXNMapper atom mapping errors
        try:
            am_rxn_sma2 = atom_map(rxn_sma2)
        except:
            kr_am_errors.append((h, z, kr[-1]))
            continue

        # Construct reaction objects
        rxns = []
        for elt in [am_rxn_sma1, am_rxn_sma2]:
            temp = AllChem.ReactionFromSmarts(elt, useSmiles=True)
            temp.Initialize()
            rxns.append(temp)

        rc_atoms = [elt.GetReactingAtoms() for elt in rxns] # Get reaction center atom idxs

        # Construct rxn ctr mol objs
        try: # REMOVE after addressing KekulizationException in get_sub_mol
            rcs = []
            for i, t_rxn in enumerate(rxns):
                temp = []
                for j, t_mol in enumerate(t_rxn.GetReactants()):
                    temp.append(get_sub_mol(t_mol, rc_atoms[i][j]))
                rcs.append(temp)
        except:
            kekulize_issues.append((h, z, kr[-1]))
            continue

        # Align substrates of the 2 reactions
        rc_idxs = [] # Each element: (idx for rxn 1, idx for rxn 2)
        remaining = [[i for i in range(len(elt))] for elt in rcs]
        while (len(remaining[0]) > 0) & (len(remaining[1]) > 0):
            idx_pair = align_substrates(rcs, remaining)

            if idx_pair is None:
                break
            else:
                rc_idxs.append(idx_pair)
                remaining[0].remove(idx_pair[0])
                remaining[1].remove(idx_pair[1])

        # Skip if you haven't aligned every reactant pred to known
        if len(rc_idxs) < len(rxn_sma1.split('>>')[0].split('.')):
            alignment_issues.append((h, z, kr[-1]))
            continue

        # For reaction 2 (known reaction) Re-order rcs, rc_atoms,
        # internal order of reactants in the reaction object in rxns
        # and the smarts stored in the known_reactions attribute of the
        # associated predicted reaction

        # Sort reaction 2 rc_idxs by reaction 1 rc_idxs
        rxn_1_rc_idxs, rxn_2_rc_idxs = list(zip(*rc_idxs))
        if rxn_1_rc_idxs != rxn_2_rc_idxs:
            rxn_2_rc_idxs, rxn_1_rc_idxs = sort_x_by_y(rxn_2_rc_idxs, rxn_1_rc_idxs)

            # Re-order atom-mapped smarts string, and then update known_rxns entry
            # with de-atom-mapped version of this string because atom mapper changes
            # reactant order and its this order that rcs, rcatoms, rc_idxs all come from
            am_ro_sma2 = am_rxn_sma2.split('>>')[0].split('.') # Get list of reactant strings
            am_ro_sma2 = '.'.join([am_ro_sma2[elt] for elt in rxn_2_rc_idxs]) # Re-join in new order
            am_rxn_sma2 = am_ro_sma2 + '>>' + am_rxn_sma2.split('>>')[1] # Join with products side

            # Re-construct reaction object from re-ordered, am smarts
            foo = rxns[1]
            temp = AllChem.ReactionFromSmarts(am_rxn_sma2, useSmiles=True)
            temp.Initialize()
            rxns[1] = temp
            bar = rxns[1]

            rc_atoms[1] = rxns[1].GetReactingAtoms() # Update rc_atoms
            rcs[1] = [get_sub_mol(elt, rc_atoms[1][i]) for i, elt in enumerate(rxns[1].GetReactants())] # Update rc mol obj
        
        pred_rxns[h].known_rxns[z][1] = rm_atom_map_num(am_rxn_sma2) # Update known_reaction entry w/ de-am smarts
        rxns = align_atom_map_nums(rxns, rcs, rc_atoms)

        # Compute MCS seeded by reaction center
        prc_mcs = get_prc_mcs(rxns, rcs, rc_atoms, norm=norm) 
        pred_rxns[h].known_rxns[z][0] = prc_mcs # Update pred_rxns
        
        a += 1 # Count known rxn analyzed
        pred_rxns[h].smarts = rm_atom_map_num(am_rxn_sma1) # Update pred_rxn smarts w/ de-am smarts

    print(x, ':', a / (z+1), 'of', z+1)

0 : 0.08139534883720931 of 86
1 : 0.6086956521739131 of 23
2 : 0.0 of 26
3 : 0.0 of 40
4 : 0.0 of 40
5 : 0.13636363636363635 of 88
6 : 0.0 of 86
7 : 0.6086956521739131 of 23
8 : 0.0 of 26
9 : 0.08139534883720931 of 86
10 : 1.0 of 7
11 : 0.0 of 86
12 : 1.0 of 7
13 : 0.15 of 40
14 : 0.0 of 26
15 : 0.723404255319149 of 47
16 : 0.975609756097561 of 41
17 : 0.6046511627906976 of 86
18 : 0.975609756097561 of 41
19 : 0.0 of 86
20 : 0.0 of 26
21 : 0.9459459459459459 of 37
22 : 0.15 of 40
23 : 0.6533333333333333 of 75
24 : 0.6086956521739131 of 23
25 : 0.961038961038961 of 77
26 : 0.95 of 20
27 : 0.08139534883720931 of 86
28 : 0.6086956521739131 of 23
29 : 0.975609756097561 of 41
30 : 0.6533333333333333 of 75
31 : 0.975609756097561 of 41
32 : 1.0 of 7
33 : 0.7368421052631579 of 19
34 : 0.6086956521739131 of 23
35 : 1.0 of 15
36 : 0.6086956521739131 of 23
37 : 0.05263157894736842 of 19
38 : 0.13636363636363635 of 88
39 : 0.15 of 40
40 : 0.0 of 40
41 : 0.723404255319149 of 47
42 : 0.15 of 40
43 :

RDKit ERROR: [22:58:29] non-ring atom 3 marked aromatic
[22:58:29] non-ring atom 3 marked aromatic


67 : 0.953757225433526 of 173
68 : 0.0 of 40
69 : 0.9090909090909091 of 11
70 : 1.0 of 1
71 : 0.15 of 40
72 : 1.0 of 10
73 : 0.15 of 40
74 : 0.4 of 20
75 : 0.9487179487179487 of 39
76 : 0.15 of 40
77 : 0.23076923076923078 of 26
78 : 1.0 of 21
79 : 1.0 of 28


RDKit ERROR: [22:59:16] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34
[22:59:16] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34

RDKit ERROR: 


81 : 0.723404255319149 of 47
82 : 1.0 of 21


In [63]:
# Thermo

# starters = 'succinate'
# targets = 'mvacid'
# generations = 4
# args = ['-s', f"{starters}", '-t', f"{targets}", '-g', str(generations)]
# command = f"source activate /home/stef/miniconda3/envs/thermo && python /home/stef/pickaxe_thermodynamics/path_mdf.py {' '.join(args)}"
# subprocess.run(command, shell=True)

thermo = load_json(thermo_dir + fn)
for k,v in thermo.items():
    st = tuple(k.split('>'))
    for i, elt in enumerate(thermo[k]):
        paths[st][i].mdf = elt['mdf']
        paths[st][i].dG_opt = elt['dG_opt']
        paths[st][i].dG_err = elt['dG_err']
        paths[st][i].conc_opt = elt['conc_opt']


In [64]:
# Save reactions dict and paths list (ultimately will replace with expansion object)

rxns_fn = 'predicted_reactions_' + fn
paths_fn = 'paths_' + fn
save_dir = '../data/processed_expansions/'
rxns_path = save_dir + rxns_fn
paths_path = save_dir + paths_fn

with open(rxns_path, 'wb') as f:
    pickle.dump(pred_rxns, f)

with open(paths_path, 'wb') as f:
    pickle.dump(paths, f)

In [65]:
len(alignment_issues), len(pr_am_errors), len(kr_am_errors)

(1854, 1, 329)